In [1]:
import cv2
import os
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import numpy as np
import mediapipe_preprocessing
import video_augmentation
import math

In [2]:
def preprocess_video(video):
    data = []
    for frame in video:
        image, results = mediapipe_preprocessing.mediapipe_detection(frame, mediapipe_preprocessing.holistic)
        landmarks = []
        if results.right_hand_landmarks != None:
            for landmark in results.right_hand_landmarks.landmark:
                landmarks = landmarks + [landmark.x, landmark.y, landmark.z]
        else:
            landmarks = landmarks + [0, 0, 0] * 21

        if results.left_hand_landmarks != None:
            for landmark in results.left_hand_landmarks.landmark:
                landmarks = landmarks + [landmark.x, landmark.y, landmark.z]
        else:
            landmarks = landmarks + [0, 0, 0] * 21

        if results.pose_landmarks != None:
            nose_landmark = results.pose_landmarks.landmark[0]
            nose_landmarks = [nose_landmark.x, nose_landmark.y, nose_landmark.z] * 59
            for i in range(0, 17):
                landmark = results.pose_landmarks.landmark[i]
                landmarks = landmarks + [landmark.x, landmark.y, landmark.z]
        else:
            nose_landmarks = [0,0,0] * 59
            landmarks = landmarks + [0, 0, 0] * 17
        data.append(np.array(landmarks) - np.array(nose_landmarks))
    return data

Train

In [5]:

dirs = [name.removesuffix('.avi') for name in os.listdir('assets/60_videos')]

train = []
labels = []
counter = 1
for type in dirs:
    print(f"{counter}. {type}")
    for folder in os.listdir('recordings'):
        files_path = f'recordings/{folder}/{type}'
        files = os.listdir(files_path)
        
        for file in files:
            vcap = cv2.VideoCapture(f'recordings/{folder}/{type}/{file}')
            frames = []
            while(True):
                ret, frame = vcap.read()
                if frame is not None:
                    frames.append(frame)
                else:
                    vcap.release()
                    cv2.destroyAllWindows()
                    break
            
            if len(frames) < 14:
                extra_num = 14-len(frames)
                last = math.ceil(extra_num/2)
                first = extra_num//2
                frames = [frames[0]]*(first) + frames + [frames[-1]]*(last)
            elif 30 > len(frames) > 14:
                frames = frames[-14:]
            if len(frames) == 30:
                frame_counter = 0
                new_frames = []
                for i in range(0, len(frames), math.floor(len(frames)/14)):
                    if frame_counter >= 14:
                        break
                    new_frames.append(frames[i])
                    frame_counter += 1
                if len(new_frames) != 14:
                    new_frames.append(frames[-1])
                frames = new_frames
            print(len(frames))
            augmented_frames = video_augmentation.augment_frames(frames)
            print('Finished augmenting frames!')
            videos = augmented_frames + [frames]
            mediapipe_preprocessed_data = []
            for video in videos:
                data = preprocess_video(video)
                mediapipe_preprocessed_data.append(data)
            
            print('Finished preprocess data!')
            word = type
            train = train + mediapipe_preprocessed_data
            labels = labels + [word] * len(mediapipe_preprocessed_data)
    print(len(train))
    counter += 1
    if counter == 1:
        break

enc = LabelEncoder()
labels = enc.fit_transform(labels)
labels = tf.keras.utils.to_categorical(labels)
train = np.array(train)
print(train.shape)







1. Ahli
14
Finished augmenting frames!


c:\Users\yosua\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
80
2. Alam
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess data!
14
Finished augmenting frames!
Finished preprocess 

In [ ]:
len(train)

In [ ]:
for elm in train:
    print(np.array(elm).shape)

Test

In [6]:
dirs = [name.removesuffix('.avi') for name in os.listdir('assets/60_videos')]

test = []
test_labels = []
count = 1
for type in dirs:
    print(f'{count}. type: {type}')
    file_path = f'assets/60_videos/{type}.avi'
    
    vcap = cv2.VideoCapture(file_path)
    frames = []
    while(True):
        ret, frame = vcap.read()
        if frame is not None:
            frames.append(frame)
        else:
            vcap.release()
            cv2.destroyAllWindows()
            break
    data = preprocess_video(frames)
    data = np.array(data)
    test.append(data)
    test_labels.append(type)
    count += 1

test = np.array(test)
test_labels = enc.transform(test_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)
test_labels

1. type: Ahli


c:\Users\yosua\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


2. type: Alam
3. type: Anda
4. type: Bahasa
5. type: Beli
6. type: Bicara
7. type: Buka
8. type: Buku
9. type: Bumi
10. type: Bunga
11. type: Burung
12. type: Cangkir
13. type: Cinta
14. type: Cuci
15. type: Daerah
16. type: Daftar
17. type: Darurat
18. type: Dia
19. type: Duduk
20. type: Ekonomi
21. type: Emosi
22. type: Film
23. type: Hadiah
24. type: Hakim
25. type: Halaman
26. type: Hewan
27. type: Hidung
28. type: Ikan
29. type: Jaket
30. type: Jam
31. type: Jual
32. type: Kaki
33. type: Kerja
34. type: Lari
35. type: Lihat
36. type: Madu
37. type: Masak
38. type: Minum
39. type: Negara
40. type: Orang
41. type: Otot
42. type: Paket
43. type: Pantai
44. type: Pintu
45. type: Pulau
46. type: Retak
47. type: Sabuk
48. type: Sapu
49. type: Saya
50. type: Tamu
51. type: Tidur
52. type: Tinggi
53. type: Topi
54. type: Tubuh
55. type: Tugas
56. type: Tuhan
57. type: Tutup
58. type: Uang
59. type: Utama
60. type: Utang


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
dirs = [name.removesuffix('.avi') for name in os.listdir('assets/60_videos')]

test = []
test_labels = []
count = 1
for type in dirs:
    print(f'{count}. type: {type}')
    file_path = f'recordings_Taufiq/{type}'
    for name in os.listdir(file_path):
        vcap = cv2.VideoCapture(file_path + f'/{name}')
        frames = []
        while(True):
            ret, frame = vcap.read()
            if frame is not None:
                frames.append(frame)
            else:
                vcap.release()
                cv2.destroyAllWindows()
                break
        if len(frames) < 14:
            extra_num = 14-len(frames)
            frames = frames + [frames[-1]]*(extra_num)
        elif len(frames) > 14:
            frames = frames[-14:]
        data = preprocess_video(frames)
        data = np.array(data)
        test.append(data)
        test_labels.append(type)
    count += 1

test = np.array(test)
test_labels = enc.transform(test_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)
test_labels

In [10]:
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.95 and logs.get('loss') < 1e-1:
            self.model.stop_training = True

model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(180, input_shape=[14, 177], return_sequences=True),
    tf.keras.layers.LSTM(180, return_sequences=True),
    tf.keras.layers.LSTM(180, return_sequences = True),
    tf.keras.layers.LSTM(180, return_sequences=True),
    tf.keras.layers.LSTM(180),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(60, activation='softmax')
])
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train, labels, epochs= 200, validation_data=(test, test_labels))
# model.fit(train, labels, epochs= 120)
model.save('test_model3.h5')

c:\Users\yosua\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 14, 180)        │       257,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 14, 180)        │       259,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 14, 180)        │       259,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 14, 180)        │       259,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 180)            │       259,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 180)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 100)            │        18,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 60)             │         6,060 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,331,700 (5.08 MB)

 Trainable params: 1,331,700 (5.08 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
151/151 ━━━━━━━━━━━━━━━━━━━━ 15s 56ms/step - accuracy: 0.0306 - loss: 3.9290 - val_accuracy: 0.0333 - val_loss: 3.5345
Epoch 2/200
151/151 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.0533 - loss: 3.5368 - val_accuracy: 0.0500 - val_loss: 3.5284
Epoch 3/200
151/151 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step - accuracy: 0.0894 - loss: 3.2179 - val_accuracy: 0.1167 - val_loss: 3.2666
Epoch 4/200
151/151 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step - accuracy: 0.1255 - loss: 3.0304 - val_accuracy: 0.0833 - val_loss: 3.2812
Epoch 5/200
151/151 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.1817 - loss: 2.6733 - val_accuracy: 0.2000 - val_loss: 2.9267
Epoch 6/200
151/151 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step - accuracy: 0.2432 - loss: 2.4109 - val_accuracy: 0.2833 - val_loss: 2.6662
Epoch 7/200
151/151 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.3351 - loss: 2.0580 - val_accuracy: 0.2833 - val_loss: 2.7466
Epoch 8/200
151/151 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.4004 - loss: 1.8665 -

KeyboardInterrupt: 

In [ ]:
model.fit(train, labels, epochs= 100, validation_data=(test, test_labels))
model.save('test_model3.h5')

In [11]:
model.save('test_model3.h5')